# RARR: Researching and Revising What Language Models Say, Using Language Models

### Warning about Hallucinate Evidence Mode
If you do not have a Bing API key, we provide an option to use RARR with evidence hallucinated from a LLM.
This mode should not be used when trying to improve attribution as the generated evidence may contain inaccuracies.
This mode is only provided to be able to quickly see how RARR works.



## Environment Setup

In [1]:
# !git clone https://github.com/anthonywchen/RARR.git
# !pip install -r RARR/requirements.txt

In [5]:
import sys
sys.path.append('..')

import json
import os
import uuid
from IPython.display import HTML, display
# sys.path.append("RARR/")

# Set your OpenAI and Bing API keys here
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["AZURE_SEARCH_KEY"] = ""

from run_editor_sequential import run_editor_one_instance
from utils import question_generation
from prompts import rarr_prompts


def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
# get_ipython().events.register('pre_run_cell', set_css)

In [2]:
claim = "Mark Hunter is a 30 year old man with a radio station in California."

In [3]:
from narrative_qa_rag import create_vecdb, retrieve_evidence
vecdb = create_vecdb()

FAISS vector store already exists, loading...


In [ ]:
col = vecdb._client.get_collection('')

## Calls RARR with Evidence Search

In [ ]:
questions = question_generation.run_rarr_question_generation(
        claim=claim,
        context=None,
        model="gpt-3.5-turbo-1106",
        prompt=rarr_prompts.RAG_QGEN_PROMPT,
        temperature=0,
        num_rounds=3
    )
print(questions)

['Age of Mark Hunter', "Location of Mark Hunter's radio station"]


In [ ]:
test_claim = questions[0]
evid = retrieve_evidence(test_claim, vecdb, top_k=2)
print(evid)

[{'text': 'Mark Hunter (Slater), a high school student in a sleepy suburb of Phoenix, Arizona, starts an FM pirate radio station that broadcasts from the basement of his parents\' house. Mark is a loner, an outsider, whose only outlet for his teenage angst and aggression is his unauthorized radio station. His pirate station\'s theme song is "Everybody Knows" by Leonard Cohen and there are glimpses of cassettes by such alternative musicians as The Jesus and Mary Chain, Camper Van Beethoven, Primal Scream, Soundgarden, Ice-T, Bad Brains, Concrete Blonde, Henry Rollins, and The Pixies. By day, Mark is seen as a loner, hardly talking to anyone around him; by night, he expresses his outsider views about what is wrong with American society. When he speaks his mind about what is going on at his school and in the community, more and more of his fellow students tune in to hear his show.', 'metadata': {'document_id': '0025577043f5090cd603c6aea60f26e236195594', 'set': 'test'}, 'query': 'Age of Ma

In [ ]:
result = run_editor_one_instance(claim=claim, model='gpt-3.5-turbo-1106', vecdb=vecdb)
print(json.dumps(result, indent=4))

Question Generation Time: 2.76 seconds
Evidence Retrieval Time: 0.07 seconds
Agreement Gate Time: 18.27 seconds
Editing Time: 0.00 seconds
Total Time: 21.36 seconds
{
    "context": null,
    "text": "Mark Hunter is a 30 year old man with a radio station in California.",
    "questions": [
        "Age of Mark Hunter",
        "Location of Mark Hunter's radio station",
        "Location of radio station of Mark Hunter",
        "Occupation of Mark Hunter"
    ],
    "evidences_for_questions": [
        [
            {
                "text": "Mark Hunter (Slater), a high school student in a sleepy suburb of Phoenix, Arizona, starts an FM pirate radio station that broadcasts from the basement of his parents' house. Mark is a loner, an outsider, whose only outlet for his teenage angst and aggression is his unauthorized radio station. His pirate station's theme song is \"Everybody Knows\" by Leonard Cohen and there are glimpses of cassettes by such alternative musicians as The Jesus and M

In [ ]:
from narrative_qa_rag import create_vecdb, retrieve_evidence

claim = "Who is chasing Mark and Nora in the jeep?"
vecdb = create_vecdb()

print(vecdb._client.list_collections()) # looks good

collection = vecdb._client.get_collection("narrative_qa_collection")
# collection.query(
#     query_texts=[claim],
#     n_results=3
# )

evidences = vecdb.similarity_search(claim, k=3)
# retriever = vecdb.as_retriever(search_type='similarity', search_kwargs={"k": 3})
# evidences = retriever.get_relevant_documents(claim)
print(evidences)
# result = run_editor_one_instance(claim=claim, model='gpt-3.5-turbo-1106')
# print(json.dumps(result, indent=4))


Chromadb already exists, loading...
[Collection(name=narrative_qa_collection), Collection(name=langchain)]
[]


In [ ]:
from utils import search_client_share
search = search_client_share.search_client
res = search('machine learning')
res['hits']

[{'itemtitle': ['A Non-technical Introduction to Machine Learning'],
  'abs': ['This chapter provides an introduction to machine learning for a non-technical readership. Machine learning is an approach to artificial intelligence. The chapter thus starts with a brief history of artificial intelligence in order to put machine learning into this broader scientific context. We then describe the main general concepts of machine learning. Readers with a background in computer science may skip this chapter.'],
  'relevance': 0.87758285,
  'pubyr': '2023',
  'numcitedby': 1},
 {'itemtitle': ['MACHINE LEARNING THEORY AND METHODS'],
  'abs': ['Machine learning alludes to the changes in frameworks that perform correlate functions connected with computing. This chapter presents introduction varieties and the application of machine learning. This chapter additionally presents the essential structures of Machine learning, the process and basic ideas associated with the varied machine learning techni

## Calls RARR with Evidence Hallucination
If you do not have a Bing Search API key but would still like to see how the RARR pipeline works, we provide an option to use a LLM from OpenAI to hallucinate evidence instead of retrieving evidence. Do NOT trust the results as the evidence can have misinformation; this is only provided to quickly illustrate how RARR operates.



In [ ]:
do_not_trust_result = run_editor_one_instance(claim, hallucinate_evidence=True)
print(json.dumps(do_not_trust_result, indent=4))